In [1]:
!pip install ultralytics
!pip install -q transformers
!pip install -q -U sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 406.7 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [2]:
from ultralytics import YOLO
from google.colab import drive
drive.mount("/content/gdrive/")


Mounted at /content/gdrive/


In [3]:
ncnn_model = YOLO('/content/gdrive/My Drive/MSc Project/Saved Models/yolov8n_ncnn_model', task='detect')

In [4]:
video_path = '/content/gdrive/MyDrive/MSc Project/IMG_0914.MOV'

In [5]:
import cv2
from google.colab.patches import cv2_imshow
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import time
import numpy as np
import re

device = 'cpu'
model = 'microsoft/trocr-base-printed'
processor = TrOCRProcessor.from_pretrained(model)
model = VisionEncoderDecoderModel.from_pretrained(model).to(device)
# ncnn_model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
import cv2

# Path to the video file
# Initialize video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not access video")

# Get the total number of frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Get the frames per second (FPS)
fps = cap.get(cv2.CAP_PROP_FPS)

# Calculate the runtime in seconds
runtime_seconds = frame_count / fps

# Convert the runtime to minutes and seconds
runtime_minutes = int(runtime_seconds // 60)
runtime_seconds = int(runtime_seconds % 60)

# Output the results
print(f"Total number of frames: {frame_count}")
print(f"Runtime of the video: {runtime_minutes} minutes and {runtime_seconds} seconds")

# Release the video capture object
cap.release()


Total number of frames: 589
Runtime of the video: 0 minutes and 19 seconds


In [10]:
import cv2
import time
import numpy as np
from PIL import Image
import re

# Initialize video capture
cap = cv2.VideoCapture(video_path, cv2.CAP_FFMPEG)
if not cap.isOpened():
    print("Error: Could not access video")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video_resized.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    original_height, original_width = frame.shape[:2]

    if frame_count % 3 == 0:
        resized_img = cv2.resize(frame, (640, 640))

        results = ncnn_model(resized_img)
        class_names = results[0].names
        boxes = results[0].boxes.xyxy.cpu().numpy()
        cls = results[0].boxes.cls

        class_colors = {
            '0': (255, 0, 0),
            '1': (0, 0, 255)
        }

        # Scaling factors
        x_scale = original_width / 640.0
        y_scale = original_height / 640.0

        # Iterate through the boxes
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box[:4]  # Extract coordinates
            conf = results[0].boxes.conf[i].item()  # Extract confidence score
            label = int(results[0].boxes.cls[i].item())  # Extract label (if available)

            # Scale coordinates back to the original image size
            x1 = int(x1 * x_scale)
            y1 = int(y1 * y_scale)
            x2 = int(x2 * x_scale)
            y2 = int(y2 * y_scale)

            if label == 1:
                cropped_img = frame[y1:y2, x1:x2]
                if cropped_img.size == 0:
                    continue  # Skip empty crops
                image = Image.fromarray(np.uint8(cropped_img)).convert('RGB')
                pixel_values = processor(image, return_tensors='pt').pixel_values.to(device)
                generated_ids = model.generate(pixel_values)
                generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

                # Remove all special characters
                txt = re.sub(r'\W+', '', generated_text).upper()

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), class_colors[str(label)], 2)
                msg = f"{class_names[int(label)]}: {txt}"
                cv2.putText(frame, msg, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, class_colors[str(label)], thickness=2)
            else:
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), class_colors[str(label)], 2)
                msg = f"{class_names[int(label)]}"
                cv2.putText(frame, msg, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, class_colors[str(label)], thickness=2)

    frame_count += 1
    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time if elapsed_time > 0 else 0
    cv2.putText(frame, f'FPS:{fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), thickness=2)

    # Uncomment the line below if you want to display the frames
    if frame_count % 20 == 0:
      cv2_imshow(frame)

    out.write(frame)

    # If you want to view the frames, add a break condition for quitting
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.

- output screenshot of both cars with the fps closer to the car

In [ ]:
from google.colab import files
files.download('output_video_resized.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>